In [9]:
!python -m spacy download en
!python -m spacy download en_core_web_sm

[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.2.0/en_core_web_sm-3.2.0-py3-none-any.whl (13.9 MB)
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


<h2> 3.6 Featurizing text data with tfidf weighted word-vectors </h2>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import re
import time
import warnings
import numpy as np
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
warnings.filterwarnings("ignore")
import sys
import os 
import pandas as pd
import numpy as np
from tqdm import tqdm

# exctract word2vec vectors
# https://github.com/explosion/spaCy/issues/1721
# http://landinghub.visualstudio.com/visual-cpp-build-tools
import spacy

from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

In [2]:
# avoid decoding problems
df = pd.read_csv("train.csv.zip")
 
# encode questions to unicode
# https://stackoverflow.com/a/6812069
# ----------------- python 2 ---------------------
# df['question1'] = df['question1'].apply(lambda x: unicode(str(x),"utf-8"))
# df['question2'] = df['question2'].apply(lambda x: unicode(str(x),"utf-8"))
# ----------------- python 3 ---------------------
df['question1'] = df['question1'].apply(lambda x: str(x))
df['question2'] = df['question2'].apply(lambda x: str(x))

In [3]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(df['question1']) + list(df['question2'])

tfidf = TfidfVectorizer(lowercase=False, )
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

- After we find TF-IDF scores, we convert each question to a weighted average of word2vec vectors by these scores.
- here we use a pre-trained GLOVE model which comes free with "Spacy".  https://spacy.io/usage/vectors-similarity
- It is trained on Wikipedia and therefore, it is stronger in terms of word semantics. 

In [5]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [7]:
def weighted_tfidf(que):
    #vecs = []
    #for que in tqdm(list(feature)):
    doc = nlp(que) 
    # 384 is the number of dimensions of vectors 
    mean_vec = np.zeros([len(doc), len(doc[0].vector)])
    for word in doc:
        # word2vec
        vec = word.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word)]
        except:
            idf = 0
        # compute final vec
        mean_vec += vec * idf
    mean_vec = mean_vec.mean(axis=0)
        #vecs.append(mean_vec)
    return mean_vec

In [ ]:
df['q1_feats_m'] = df['question1'].progress_apply(weighted_tfidf)
df['q2_feats_m'] = df['question2'].progress_apply(weighted_tfidf)

In [90]:
#prepro_features_train.csv (Simple Preprocessing Feartures)
#nlp_features_train.csv (NLP Features)
if os.path.isfile('nlp_features_train.csv'):
    dfnlp = pd.read_csv("nlp_features_train.csv",encoding='latin-1')
else:
    print("download nlp_features_train.csv from drive or run previous notebooks")

if os.path.isfile('train_before_preprocessing.csv'):
    dfppro = pd.read_csv("train_before_preprocessing.csv",encoding='latin-1')
else:
    print("download train_before_preprocessing.csv from drive or run previous notebook")

In [91]:
df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)
df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3 = df.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)
df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)
df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)

In [92]:
# dataframe of nlp features
df1.head()

,id,is_duplicate,cwc_min,cwc_max,csc_min,csc_max,ctc_min,ctc_max,last_word_eq,first_word_eq,abs_len_diff,mean_len,token_set_ratio,token_sort_ratio,fuzz_ratio,fuzz_partial_ratio,longest_substr_ratio
0,0,0,0.999980,0.833319,0.999983,0.999983,0.916659,0.785709,0.0,1.0,2.0,13.0,100,92,92,100,0.982143
1,1,0,0.666644,0.249997,0.666644,0.499988,0.499994,0.307690,0.0,1.0,5.0,10.5,81,64,66,70,0.553191
2,2,0,0.599988,0.499992,0.399992,0.249997,0.499995,0.357140,0.0,1.0,4.0,12.0,73,64,41,38,0.188679
3,3,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,4.0,9.0,30,32,24,39,0.088235
4,4,0,0.399992,0.199998,0.999950,0.666644,0.571420,0.307690,0.0,1.0,6.0,10.0,68,49,36,54,0.157895


In [93]:
# data before preprocessing 
df2.head()

,id,freq_qid1,freq_qid2,q1len,q2len,q1_n_words,q2_n_words,word_Common,word_Total,word_share,freq_q1+q2,freq_q1-q2
0,0,1,1,66,57,14,12,10.0,23.0,0.434783,2,0
1,1,4,1,51,88,8,13,4.0,20.0,0.200000,5,3
2,2,1,1,73,59,14,10,4.0,24.0,0.166667,2,0
3,3,1,1,50,65,11,9,0.0,19.0,0.000000,2,0
4,4,3,1,76,39,13,7,2.0,20.0,0.100000,4,2


In [94]:
# Questions 1 tfidf weighted word2vec
df3_q1.head()

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,18.807843,41.964926,33.081681,4.244238,43.027659,-10.013383,-10.744927,-11.815859,5.588288,1.115522,...,20.576247,-21.568797,-24.228453,14.157065,-16.261115,25.544021,25.480808,39.165071,3.834185,9.871860
1,-23.079738,14.342906,-0.805506,-19.245326,8.865776,-9.310454,-16.653165,-13.512745,3.500085,-3.910091,...,53.702352,13.131150,-22.455495,-54.181957,0.183353,-3.724258,-12.677888,19.355100,11.162901,19.004885
2,-0.112307,18.152627,17.287431,23.381115,25.388425,-6.092771,-20.567529,-15.121522,-2.940608,16.324171,...,-2.378785,-2.544791,-10.461190,16.518055,-21.963829,22.504434,2.132555,8.428122,27.520017,-22.875334
3,-22.978012,3.542216,-21.203833,6.316206,11.723298,11.653462,-29.423875,2.754801,-17.613957,-12.570249,...,-25.697054,-17.881601,-30.913518,1.010653,8.655211,22.253350,-7.253639,18.266633,51.872220,6.351798
4,47.606715,76.045652,-16.103146,25.784890,4.649403,-14.228687,-42.204426,32.545716,4.067297,-16.961617,...,22.576464,-35.968549,-19.760356,-52.964732,-60.509826,20.788706,45.909070,44.480312,-28.241497,18.110361


In [95]:
# Questions 2 tfidf weighted word2vec
df3_q2.head()

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,18.710824,37.375986,45.405516,5.426995,33.751131,-3.253420,-4.959911,-13.442803,3.905695,4.777382,...,23.793560,-20.588538,-25.390728,12.380474,-19.671300,16.618838,30.438979,43.384339,5.521896,2.243536
1,-13.239171,38.161040,31.628311,-9.844968,17.468515,-19.613922,-16.775893,1.972479,-5.051891,6.606223,...,68.032528,1.352752,-25.395078,-7.580498,-23.430201,3.710806,1.435839,44.218947,11.410592,22.943373
2,1.774256,41.910379,11.486097,-2.761912,31.372257,-0.054157,-33.715000,9.834838,-8.615697,-18.969312,...,-10.696379,-35.324791,-20.233565,40.031800,-22.515539,12.892510,0.084740,21.583779,24.523905,-7.576786
3,5.639871,15.257331,8.393493,34.085459,17.597781,2.442303,-16.274006,0.360628,-10.255298,-14.794721,...,6.891580,-21.247972,0.394724,41.386917,-9.613216,18.265161,-1.723291,1.429064,18.121417,18.179006
4,9.878437,30.031709,29.496887,0.223323,16.194255,7.182570,-18.419677,6.949175,11.067374,7.980174,...,17.671461,-11.538394,-11.433049,14.088319,-32.659020,6.244056,20.535059,17.647714,12.484857,-12.859808


In [96]:
print("Number of features in nlp dataframe :", df1.shape[1])
print("Number of features in preprocessed dataframe :", df2.shape[1])
print("Number of features in question1 w2v  dataframe :", df3_q1.shape[1])
print("Number of features in question2 w2v  dataframe :", df3_q2.shape[1])
print("Number of features in final dataframe  :", df1.shape[1]+df2.shape[1]+df3_q1.shape[1]+df3_q2.shape[1])

Number of features in nlp dataframe : 17
Number of features in preprocessed dataframe : 12
Number of features in question1 w2v  dataframe : 96
Number of features in question2 w2v  dataframe : 96
Number of features in final dataframe  : 221


In [97]:
# storing the final features to csv file
if not os.path.isfile('final_features.csv'):
    df3_q1['id']=df1['id']
    df3_q2['id']=df1['id']
    df1  = df1.merge(df2, on='id',how='left')
    df2  = df3_q1.merge(df3_q2, on='id',how='left')
    result  = df1.merge(df2, on='id',how='left')
    result.to_csv('final_features.csv', index = False)

In [98]:
list(result.columns)

['id',
 'is_duplicate',
 'cwc_min',
 'cwc_max',
 'csc_min',
 'csc_max',
 'ctc_min',
 'ctc_max',
 'last_word_eq',
 'first_word_eq',
 'abs_len_diff',
 'mean_len',
 'token_set_ratio',
 'token_sort_ratio',
 'fuzz_ratio',
 'fuzz_partial_ratio',
 'longest_substr_ratio',
 'freq_qid1',
 'freq_qid2',
 'q1len',
 'q2len',
 'q1_n_words',
 'q2_n_words',
 'word_Common',
 'word_Total',
 'word_share',
 'freq_q1+q2',
 'freq_q1-q2',
 '0_x',
 '1_x',
 '2_x',
 '3_x',
 '4_x',
 '5_x',
 '6_x',
 '7_x',
 '8_x',
 '9_x',
 '10_x',
 '11_x',
 '12_x',
 '13_x',
 '14_x',
 '15_x',
 '16_x',
 '17_x',
 '18_x',
 '19_x',
 '20_x',
 '21_x',
 '22_x',
 '23_x',
 '24_x',
 '25_x',
 '26_x',
 '27_x',
 '28_x',
 '29_x',
 '30_x',
 '31_x',
 '32_x',
 '33_x',
 '34_x',
 '35_x',
 '36_x',
 '37_x',
 '38_x',
 '39_x',
 '40_x',
 '41_x',
 '42_x',
 '43_x',
 '44_x',
 '45_x',
 '46_x',
 '47_x',
 '48_x',
 '49_x',
 '50_x',
 '51_x',
 '52_x',
 '53_x',
 '54_x',
 '55_x',
 '56_x',
 '57_x',
 '58_x',
 '59_x',
 '60_x',
 '61_x',
 '62_x',
 '63_x',
 '64_x',
 '65_x